In [0]:
# ============================================================================
# EUROPEAN POWER GRID STRESS PREDICTION - 6-TARGET SYSTEM
# Cell 1: Environment Setup and Library Imports
# ============================================================================

# Install required packages
# XGBoost is not included in Databricks by default, must install explicitly
import sys
!{sys.executable} -m pip install xgboost --quiet

# Core Data Processing
import pandas as pd
import numpy as np
from datetime import datetime
import warnings

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# File Management
import pickle
import json
import os

# ============================================================================
# Configuration Settings
# ============================================================================

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set visualization defaults
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Configure pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

# ============================================================================
# Verification
# ============================================================================

print("All libraries imported successfully")
print(f"Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*70)
print("ENVIRONMENT READY - PROCEED TO DATA LOADING")
print("="*70)

In [0]:
# ============================================================================
# Cell 2: Data Loading and Initial Exploration
# ============================================================================

# Load the three datasets from Databricks tables
# These splits were created previously with temporal separation to prevent leakage
print("\nLoading datasets from Databricks tables...")
print("-" * 70)

train_df = spark.table("workspace.default.train_set").toPandas()
val_df = spark.table("workspace.default.validation_set").toPandas()
test_df = spark.table("workspace.default.test_set").toPandas()

# Display dataset sizes
print(f"\nDataset sizes:")
print(f"  Training set:   {len(train_df):>8,} records")
print(f"  Validation set: {len(val_df):>8,} records")
print(f"  Test set:       {len(test_df):>8,} records")
print(f"  Total:          {len(train_df) + len(val_df) + len(test_df):>8,} records")

# Display column information
print(f"\nNumber of columns: {len(train_df.columns)}")
print(f"\nColumn names:")
print(train_df.columns.tolist())

# Display basic info about the training set
print("\n" + "="*70)
print("TRAINING SET OVERVIEW")
print("="*70)
print("\nFirst 5 rows:")
print(train_df.head())

print("\nData types:")
print(train_df.dtypes)

print("\nBasic statistics:")
print(train_df.describe())

# Check for missing values
print("\n" + "="*70)
print("MISSING VALUES ANALYSIS")
print("="*70)
missing_counts = train_df.isnull().sum()
missing_pct = (missing_counts / len(train_df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_counts,
    'Missing_Percentage': missing_pct
}).sort_values('Missing_Percentage', ascending=False)

print("\nColumns with missing values:")
print(missing_df[missing_df['Missing_Count'] > 0])

# Check temporal coverage
print("\n" + "="*70)
print("TEMPORAL COVERAGE")
print("="*70)

# Convert datetime columns if they exist
if 'datetime' in train_df.columns:
    train_df['datetime'] = pd.to_datetime(train_df['datetime'])
    val_df['datetime'] = pd.to_datetime(val_df['datetime'])
    test_df['datetime'] = pd.to_datetime(test_df['datetime'])
    
    print(f"\nTraining set:   {train_df['datetime'].min()} to {train_df['datetime'].max()}")
    print(f"Validation set: {val_df['datetime'].min()} to {val_df['datetime'].max()}")
    print(f"Test set:       {test_df['datetime'].min()} to {test_df['datetime'].max()}")

# Check country distribution
if 'country' in train_df.columns:
    print("\n" + "="*70)
    print("COUNTRY DISTRIBUTION")
    print("="*70)
    
    country_counts = train_df['country'].value_counts().sort_index()
    print(f"\nNumber of countries: {len(country_counts)}")
    print(f"\nCountries: {sorted(train_df['country'].unique())}")
    print(f"\nRecords per country (training set):")
    print(country_counts)

print("\n" + "="*70)
print("DATA LOADING COMPLETE")
print("="*70)

In [0]:
# ============================================================================
# Cell 3: Correlation Analysis and Key Feature Exploration
# ============================================================================

# Focus on the key columns we'll use for our 6-target system
# We'll analyze correlations with grid_stress_score and between key features

print("\n" + "="*70)
print("CORRELATION ANALYSIS - KEY FEATURES FOR 6-TARGET SYSTEM")
print("="*70)

# Define key features for our analysis
# These are the complete, non-missing columns most relevant to grid stress
key_features = [
    'Actual_Load',
    'Forecasted_Load', 
    'net_imports',
    'mean_temperature_c',
    'mean_wind_speed',
    'mean_ssrd',
    'reserve_margin_ml',
    'forecast_load_error',
    'load_rel_error',
    'grid_stress_score'
]

# Create subset with key features
analysis_df = train_df[key_features].copy()

print(f"\nAnalyzing {len(key_features)} key features")
print(f"Records: {len(analysis_df):,}")
print("\nFeatures selected:")
for i, feat in enumerate(key_features, 1):
    print(f"  {i}. {feat}")

# Calculate correlation matrix
print("\n" + "-"*70)
print("CORRELATION MATRIX")
print("-"*70)

correlation_matrix = analysis_df.corr()
print("\nFull correlation matrix:")
print(correlation_matrix.round(3))

# Focus on correlations with grid_stress_score
print("\n" + "-"*70)
print("CORRELATIONS WITH EXISTING GRID_STRESS_SCORE")
print("-"*70)

stress_correlations = correlation_matrix['grid_stress_score'].sort_values(ascending=False)
print("\nCorrelation with grid_stress_score (sorted):")
for feature, corr in stress_correlations.items():
    if feature != 'grid_stress_score':
        print(f"  {feature:<25} {corr:>7.4f}")

# Analyze the existing targets in the dataset
print("\n" + "="*70)
print("EXISTING TARGET ANALYSIS")
print("="*70)

print("\nExisting grid_stress_score distribution:")
print(train_df['grid_stress_score'].describe())

print("\nUnique stress score values:")
unique_scores = sorted(train_df['grid_stress_score'].unique())
print(unique_scores)

print("\nDistribution of stress scores:")
score_dist = train_df['grid_stress_score'].value_counts().sort_index()
for score, count in score_dist.items():
    pct = (count / len(train_df)) * 100
    print(f"  Score {score:>5.1f}: {count:>7,} records ({pct:>5.2f}%)")

# Analyze existing binary targets
print("\n" + "-"*70)
print("EXISTING BINARY TARGETS")
print("-"*70)

print("\nT7 (High Exports) distribution:")
print(f"  T7 = 0: {(train_df['T7_high_exports']==0).sum():>7,} records ({(train_df['T7_high_exports']==0).mean()*100:>5.2f}%)")
print(f"  T7 = 1: {(train_df['T7_high_exports']==1).sum():>7,} records ({(train_df['T7_high_exports']==1).mean()*100:>5.2f}%)")

print("\nT8 (High Imports) distribution:")
print(f"  T8 = 0: {(train_df['T8_high_imports']==0).sum():>7,} records ({(train_df['T8_high_imports']==0).mean()*100:>5.2f}%)")
print(f"  T8 = 1: {(train_df['T8_high_imports']==1).sum():>7,} records ({(train_df['T8_high_imports']==1).mean()*100:>5.2f}%)")

# Check P10/P90 thresholds for import/export
print("\n" + "-"*70)
print("IMPORT/EXPORT THRESHOLDS")
print("-"*70)

print(f"\nP10 (high exports threshold): {train_df['P10_net'].iloc[0]:,.2f} MW")
print(f"P90 (high imports threshold): {train_df['P90_net'].iloc[0]:,.2f} MW")

print("\nNet imports distribution:")
print(train_df['net_imports'].describe())

# Visualize key relationships
print("\n" + "="*70)
print("VISUALIZING KEY RELATIONSHIPS")
print("="*70)

# Create correlation heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap of all correlations
import seaborn as sns
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, ax=axes[0], cbar_kws={'label': 'Correlation'})
axes[0].set_title('Correlation Matrix - Key Features', fontsize=14, fontweight='bold')

# Bar plot of correlations with grid_stress_score
stress_corr_plot = stress_correlations.drop('grid_stress_score')
axes[1].barh(range(len(stress_corr_plot)), stress_corr_plot.values, color='steelblue')
axes[1].set_yticks(range(len(stress_corr_plot)))
axes[1].set_yticklabels(stress_corr_plot.index)
axes[1].set_xlabel('Correlation Coefficient', fontsize=12)
axes[1].set_title('Correlation with Grid Stress Score', fontsize=14, fontweight='bold')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("CORRELATION ANALYSIS COMPLETE")
print("="*70)

In [0]:
# ============================================================================
# Cell 3B: Missing Values Analysis and Handling Strategy
# ============================================================================

print("\n" + "="*70)
print("MISSING VALUES ANALYSIS FOR 6-TARGET SYSTEM")
print("="*70)

# Check missing values in key columns we'll use for our 6-target system
key_columns_for_targets = [
    'Actual_Load',
    'Forecasted_Load',
    'net_imports',
    'mean_temperature_c',
    'mean_wind_speed',
    'mean_ssrd'
]

print("\nMissing values in KEY columns for 6-target system:")
print("-" * 70)

for col in key_columns_for_targets:
    missing_count = train_df[col].isnull().sum()
    missing_pct = (missing_count / len(train_df)) * 100
    print(f"  {col:<30} {missing_count:>8,} ({missing_pct:>6.2f}%)")

# Check if we have complete records for target calculation
complete_mask = train_df[key_columns_for_targets].notna().all(axis=1)
complete_count = complete_mask.sum()
complete_pct = (complete_count / len(train_df)) * 100

print(f"\n{'='*70}")
print(f"Records with ALL key features complete: {complete_count:,} ({complete_pct:.2f}%)")
print(f"Records with ANY missing values:        {(~complete_mask).sum():,} ({(~complete_mask).mean()*100:.2f}%)")

# Check generation columns (many have missing values)
generation_cols = [col for col in train_df.columns if '__Actual_Aggregated' in col]
print(f"\n{'='*70}")
print("GENERATION COLUMNS ANALYSIS")
print("-" * 70)
print(f"\nTotal generation columns: {len(generation_cols)}")

gen_missing = []
for col in generation_cols[:10]:  # Show first 10
    missing_count = train_df[col].isnull().sum()
    missing_pct = (missing_count / len(train_df)) * 100
    gen_missing.append((col, missing_count, missing_pct))
    print(f"  {col:<50} {missing_pct:>6.2f}% missing")

print("\n  ... (showing first 10 of {}) ...".format(len(generation_cols)))

# STRATEGY DECISION
print(f"\n{'='*70}")
print("MISSING VALUES HANDLING STRATEGY")
print("="*70)

print("\nFor our 6-target stress score system:")
print("\n1. REQUIRED COLUMNS (must be complete):")
print("   - Actual_Load, Forecasted_Load, net_imports")
print("   - These are ESSENTIAL for calculating all 6 targets")
print("   - Action: DROP rows with missing values in these columns")

print("\n2. WEATHER COLUMNS (useful but not critical):")
print("   - mean_temperature_c, mean_wind_speed, mean_ssrd")
print("   - Used for feature engineering but not for targets")
print("   - Action: FILL missing values with median/mean per country")

print("\n3. GENERATION COLUMNS (44-100% missing):")
print("   - Many generation columns have extensive missing data")
print("   - NOT required for our 6-target system")
print("   - Action: DO NOT USE for initial modeling")

# Apply cleaning to training data
print(f"\n{'='*70}")
print("APPLYING CLEANING STRATEGY")
print("="*70)

# Check critical columns for targets
critical_cols = ['Actual_Load', 'Forecasted_Load', 'net_imports']

print(f"\nBefore cleaning: {len(train_df):,} records")

# Check missing in critical columns
for col in critical_cols:
    missing = train_df[col].isnull().sum()
    print(f"  {col}: {missing:,} missing")

if train_df[critical_cols].isnull().any().any():
    print("\nWARNING: Found missing values in critical columns!")
    print("Action: Will drop these rows before target calculation")
    
    clean_df = train_df.dropna(subset=critical_cols)
    print(f"\nAfter dropping rows with missing critical values: {len(clean_df):,} records")
    print(f"Dropped: {len(train_df) - len(clean_df):,} records ({(len(train_df) - len(clean_df))/len(train_df)*100:.2f}%)")
else:
    print("\nGOOD: No missing values in critical columns!")
    print("All records can be used for target calculation")
    clean_df = train_df.copy()

# Check weather columns
weather_cols = ['mean_temperature_c', 'mean_wind_speed', 'mean_ssrd']
print(f"\n{'='*70}")
print("WEATHER COLUMNS CHECK")
print("-" * 70)

for col in weather_cols:
    missing = clean_df[col].isnull().sum()
    print(f"  {col}: {missing:,} missing ({missing/len(clean_df)*100:.2f}%)")
    
if clean_df[weather_cols].isnull().any().any():
    print("\nAction: Will fill weather missing values with country medians")
    print("        (This preserves country-specific patterns)")
else:
    print("\nGOOD: No missing values in weather columns!")

print(f"\n{'='*70}")
print("SUMMARY")
print("="*70)
print(f"\nFinal usable records: {len(clean_df):,}")
print(f"Percentage retained: {len(clean_df)/len(train_df)*100:.2f}%")
print("\nReady to proceed with 6-target system creation")

In [0]:
# ============================================================================
# Cell 4: Build 6-Target Stress Score System
# ============================================================================

# Based on our project requirements, we'll create 6 operational targets
# Each target represents a specific grid stress condition with weighted impact

print("\n" + "="*70)
print("BUILDING 6-TARGET STRESS SCORE SYSTEM")
print("="*70)

# Work with a copy of training data
df_targets = train_df.copy()

# Calculate additional features needed for targets
df_targets['forecast_error_pct'] = (
    (df_targets['Actual_Load'] - df_targets['Forecasted_Load']).abs() / 
    df_targets['Forecasted_Load']
)

# Calculate percentiles for import/export thresholds
p10_imports = df_targets['net_imports'].quantile(0.10)
p90_imports = df_targets['net_imports'].quantile(0.90)
p90_magnitude = df_targets['net_imports'].abs().quantile(0.90)

print("\nImport/Export Thresholds (from training data):")
print(f"  P10 (high exports):      {p10_imports:>10,.2f} MW (negative = exporting)")
print(f"  P90 (high imports):      {p90_imports:>10,.2f} MW (positive = importing)")
print(f"  P90 magnitude (extreme): {p90_magnitude:>10,.2f} MW")

# ============================================================================
# Define the 6 Targets
# ============================================================================

print("\n" + "-"*70)
print("TARGET DEFINITIONS")
print("-"*70)

# TARGET 1: Large Forecast Error (>10%) - CRITICAL
# When actual demand differs from forecast by more than 10%
df_targets['T1_large_error'] = (df_targets['forecast_error_pct'] > 0.10).astype(int)

print("\nT1 - Large Forecast Error (>10%)")
print(f"  Weight: 25 points")
print(f"  Logic: |Actual_Load - Forecasted_Load| / Forecasted_Load > 0.10")
print(f"  Triggered: {df_targets['T1_large_error'].sum():,} records ({df_targets['T1_large_error'].mean()*100:.2f}%)")

# TARGET 2: Medium Forecast Error (5-10%) - MODERATE
# When forecast error is between 5% and 10%
df_targets['T2_medium_error'] = (
    (df_targets['forecast_error_pct'] > 0.05) & 
    (df_targets['forecast_error_pct'] <= 0.10)
).astype(int)

print("\nT2 - Medium Forecast Error (5-10%)")
print(f"  Weight: 10 points")
print(f"  Logic: 0.05 < |Actual_Load - Forecasted_Load| / Forecasted_Load ≤ 0.10")
print(f"  Triggered: {df_targets['T2_medium_error'].sum():,} records ({df_targets['T2_medium_error'].mean()*100:.2f}%)")

# TARGET 3: Underestimated Demand (>5% underforecast) - HIGH
# When actual demand exceeds forecast by more than 5%
df_targets['T3_underestimated'] = (
    ((df_targets['Actual_Load'] - df_targets['Forecasted_Load']) / df_targets['Forecasted_Load'] > 0.05) &
    (df_targets['Forecasted_Load'] < df_targets['Actual_Load'])
).astype(int)

print("\nT3 - Underestimated Demand (>5% underforecast)")
print(f"  Weight: 20 points")
print(f"  Logic: (Actual_Load - Forecasted_Load) / Forecasted_Load > 0.05 AND Actual > Forecast")
print(f"  Triggered: {df_targets['T3_underestimated'].sum():,} records ({df_targets['T3_underestimated'].mean()*100:.2f}%)")

# TARGET 7: High Exports (<P10) - MODERATE
# When country is exporting heavily (negative net_imports below 10th percentile)
df_targets['T7_high_exports_new'] = (df_targets['net_imports'] < p10_imports).astype(int)

print("\nT7 - High Exports (<P10)")
print(f"  Weight: 10 points")
print(f"  Logic: net_imports < P10 ({p10_imports:.2f} MW)")
print(f"  Triggered: {df_targets['T7_high_exports_new'].sum():,} records ({df_targets['T7_high_exports_new'].mean()*100:.2f}%)")

# TARGET 8: High Imports (>P90) - HIGH
# When country is importing heavily (positive net_imports above 90th percentile)
df_targets['T8_high_imports_new'] = (df_targets['net_imports'] > p90_imports).astype(int)

print("\nT8 - High Imports (>P90)")
print(f"  Weight: 20 points")
print(f"  Logic: net_imports > P90 ({p90_imports:.2f} MW)")
print(f"  Triggered: {df_targets['T8_high_imports_new'].sum():,} records ({df_targets['T8_high_imports_new'].mean()*100:.2f}%)")

# TARGET 9: Extreme Import/Export Flow (|imports| >P90) - HIGH
# When absolute value of imports/exports exceeds 90th percentile (stress in either direction)
df_targets['T9_extreme_flow'] = (df_targets['net_imports'].abs() > p90_magnitude).astype(int)

print("\nT9 - Extreme Import/Export Flow (|imports| >P90)")
print(f"  Weight: 15 points")
print(f"  Logic: |net_imports| > P90 ({p90_magnitude:.2f} MW)")
print(f"  Triggered: {df_targets['T9_extreme_flow'].sum():,} records ({df_targets['T9_extreme_flow'].mean()*100:.2f}%)")

# ============================================================================
# Calculate Our Stress Score
# ============================================================================

print("\n" + "="*70)
print("CALCULATING 6-TARGET STRESS SCORE")
print("="*70)

df_targets['stress_score_6target'] = (
    df_targets['T1_large_error'] * 25 +
    df_targets['T2_medium_error'] * 10 +
    df_targets['T3_underestimated'] * 20 +
    df_targets['T7_high_exports_new'] * 10 +
    df_targets['T8_high_imports_new'] * 20 +
    df_targets['T9_extreme_flow'] * 15
)

print("\nFormula: stress_score = T1×25 + T2×10 + T3×20 + T7×10 + T8×20 + T9×15")
print(f"Range: 0 to 100 points")

print("\n6-Target Stress Score Distribution:")
print(df_targets['stress_score_6target'].describe())

print("\nUnique stress score values:")
unique_new_scores = sorted(df_targets['stress_score_6target'].unique())
print(unique_new_scores[:20])  # Show first 20 values

print("\nDistribution of stress scores:")
new_score_dist = df_targets['stress_score_6target'].value_counts().sort_index()
for score, count in new_score_dist.head(15).items():
    pct = (count / len(df_targets)) * 100
    print(f"  Score {score:>3.0f}: {count:>7,} records ({pct:>5.2f}%)")

# Calculate blackout risk threshold
# Recommend threshold at 30 points
df_targets['blackout_risk_6target'] = (df_targets['stress_score_6target'] >= 30).astype(int)

print(f"\nBlackout Risk Classification (threshold = 30 points):")
print(f"  Normal (0-29):   {(df_targets['stress_score_6target'] < 30).sum():,} records ({(df_targets['stress_score_6target'] < 30).mean()*100:.2f}%)")
print(f"  At Risk (≥30):   {(df_targets['stress_score_6target'] >= 30).sum():,} records ({(df_targets['stress_score_6target'] >= 30).mean()*100:.2f}%)")

# ============================================================================
# Compare with Existing Grid Stress Score
# ============================================================================

print("\n" + "="*70)
print("COMPARISON: OUR 6-TARGET vs EXISTING GRID_STRESS_SCORE")
print("="*70)

# Calculate correlation between the two systems
correlation = df_targets['stress_score_6target'].corr(df_targets['grid_stress_score'])

print(f"\nCorrelation between systems: {correlation:.4f}")

print("\nComparison Summary:")
print(f"{'Metric':<30} {'Our 6-Target':<20} {'Existing Score':<20}")
print("-" * 70)
print(f"{'Mean':<30} {df_targets['stress_score_6target'].mean():<20.2f} {df_targets['grid_stress_score'].mean():<20.2f}")
print(f"{'Median':<30} {df_targets['stress_score_6target'].median():<20.2f} {df_targets['grid_stress_score'].median():<20.2f}")
print(f"{'Std Dev':<30} {df_targets['stress_score_6target'].std():<20.2f} {df_targets['grid_stress_score'].std():<20.2f}")
print(f"{'Min':<30} {df_targets['stress_score_6target'].min():<20.0f} {df_targets['grid_stress_score'].min():<20.0f}")
print(f"{'Max':<30} {df_targets['stress_score_6target'].max():<20.0f} {df_targets['grid_stress_score'].max():<20.0f}")
print(f"{'% Zero Stress':<30} {(df_targets['stress_score_6target']==0).mean()*100:<20.2f} {(df_targets['grid_stress_score']==0).mean()*100:<20.2f}")

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribution comparison
axes[0, 0].hist(df_targets['stress_score_6target'], bins=50, alpha=0.6, 
                label='Our 6-Target Score', color='blue', edgecolor='black')
axes[0, 0].hist(df_targets['grid_stress_score'], bins=50, alpha=0.6, 
                label='Existing Score', color='red', edgecolor='black')
axes[0, 0].axvline(30, color='orange', linestyle='--', linewidth=2, label='Blackout Threshold (30)')
axes[0, 0].set_xlabel('Stress Score', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Score Distribution Comparison', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Scatter plot: Our score vs Existing score
axes[0, 1].scatter(df_targets['grid_stress_score'], df_targets['stress_score_6target'], 
                   alpha=0.1, s=1, color='steelblue')
axes[0, 1].plot([0, 75], [0, 75], 'r--', linewidth=2, label='Perfect Agreement')
axes[0, 1].set_xlabel('Existing Grid Stress Score', fontsize=12)
axes[0, 1].set_ylabel('Our 6-Target Stress Score', fontsize=12)
axes[0, 1].set_title(f'Score Comparison (r={correlation:.3f})', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Target contribution breakdown
target_contributions = {
    'T1 (Large Error)': df_targets['T1_large_error'].sum(),
    'T2 (Medium Error)': df_targets['T2_medium_error'].sum(),
    'T3 (Underestimated)': df_targets['T3_underestimated'].sum(),
    'T7 (High Exports)': df_targets['T7_high_exports_new'].sum(),
    'T8 (High Imports)': df_targets['T8_high_imports_new'].sum(),
    'T9 (Extreme Flow)': df_targets['T9_extreme_flow'].sum()
}

axes[1, 0].barh(list(target_contributions.keys()), 
                [v/len(df_targets)*100 for v in target_contributions.values()],
                color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Percentage of Records Triggered (%)', fontsize=12)
axes[1, 0].set_title('Target Trigger Rates', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Box plots comparison
box_data = [df_targets['stress_score_6target'], df_targets['grid_stress_score']]
axes[1, 1].boxplot(box_data, labels=['Our 6-Target', 'Existing Score'])
axes[1, 1].set_ylabel('Stress Score', fontsize=12)
axes[1, 1].set_title('Score Distribution (Box Plot)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("6-TARGET SYSTEM COMPLETE")
print("="*70)
print("\nKey Findings:")
print(f"  1. Our system has correlation of {correlation:.4f} with existing score")
print(f"  2. Our system is more selective: {(df_targets['stress_score_6target']==0).mean()*100:.1f}% normal vs {(df_targets['grid_stress_score']==0).mean()*100:.1f}%")
print(f"  3. Blackout risk rate at threshold 30: {(df_targets['stress_score_6target'] >= 30).mean()*100:.1f}%")
print("\nConclusion: Our 6-target system differs significantly from existing score.")
print("We will proceed with OUR system for modeling.")

In [0]:
# ============================================================================
# Cell 5: Feature Engineering - Base Features and Lag Features
# ============================================================================

print("\n" + "="*70)
print("FEATURE ENGINEERING FOR 6-TARGET STRESS PREDICTION")
print("="*70)

# We'll engineer features on all three datasets
# Start with training data to demonstrate, then apply to val and test

def engineer_features(df, dataset_name="Dataset"):
    """
    Create comprehensive feature set including base features and lag features.
    Lag features capture temporal persistence in grid stress patterns.
    """
    
    print(f"\n{dataset_name}:")
    print("-" * 70)
    print(f"Initial records: {len(df):,}")
    
    # Create working copy
    df_feat = df.copy()
    
    # Ensure datetime column exists and is properly formatted
    if 'index' in df_feat.columns:
        df_feat['datetime'] = pd.to_datetime(df_feat['index'])
    elif 'datetime' not in df_feat.columns:
        print("ERROR: No datetime column found!")
        return df_feat
    
    # Sort by country and datetime for proper lag calculation
    df_feat = df_feat.sort_values(['country', 'datetime']).reset_index(drop=True)
    
    # ========================================================================
    # BASE FEATURES (25 features)
    # ========================================================================
    
    print("\nCreating base features...")
    
    # Load-related features
    df_feat['load_difference'] = df_feat['Actual_Load'] - df_feat['Forecasted_Load']
    df_feat['load_forecast_ratio'] = df_feat['Actual_Load'] / df_feat['Forecasted_Load']
    
    # Import/export features
    df_feat['import_magnitude'] = df_feat['net_imports'].abs()
    df_feat['import_dependency_ratio'] = df_feat['net_imports'] / df_feat['Actual_Load']
    df_feat['is_importing'] = (df_feat['net_imports'] > 0).astype(int)
    df_feat['is_exporting'] = (df_feat['net_imports'] < 0).astype(int)
    
    # Weather interaction features
    df_feat['temp_load_interaction'] = df_feat['mean_temperature_c'] * df_feat['Actual_Load']
    df_feat['wind_load_interaction'] = df_feat['mean_wind_speed'] * df_feat['Actual_Load']
    df_feat['solar_load_interaction'] = df_feat['mean_ssrd'] * df_feat['Actual_Load']
    
    # Temporal features - Cyclical encoding
    df_feat['hour'] = df_feat['datetime'].dt.hour
    df_feat['month'] = df_feat['datetime'].dt.month
    df_feat['day_of_week'] = df_feat['datetime'].dt.dayofweek
    
    # Convert to cyclical features (preserves circular nature of time)
    df_feat['hour_sin'] = np.sin(2 * np.pi * df_feat['hour'] / 24)
    df_feat['hour_cos'] = np.cos(2 * np.pi * df_feat['hour'] / 24)
    df_feat['month_sin'] = np.sin(2 * np.pi * df_feat['month'] / 12)
    df_feat['month_cos'] = np.cos(2 * np.pi * df_feat['month'] / 12)
    df_feat['dow_sin'] = np.sin(2 * np.pi * df_feat['day_of_week'] / 7)
    df_feat['dow_cos'] = np.cos(2 * np.pi * df_feat['day_of_week'] / 7)
    
    # Temporal features - Binary indicators
    df_feat['is_weekend'] = (df_feat['day_of_week'] >= 5).astype(int)
    df_feat['is_peak_hour'] = df_feat['hour'].isin([8, 9, 10, 18, 19, 20]).astype(int)
    
    print(f"  Created 25 base features")
    
    # ========================================================================
    # LAG FEATURES (18 features)
    # Critical for capturing temporal persistence in grid stress
    # ========================================================================
    
    print("\nCreating lag features (per country to prevent leakage)...")
    
    # Initialize lag feature columns
    lag_features = [
        'load_lag_1h', 'load_lag_3h', 'load_lag_6h', 'load_lag_24h',
        'import_lag_1h', 'import_lag_3h', 'import_lag_24h',
        'load_rolling_mean_6h', 'load_rolling_std_6h', 'load_rolling_mean_24h',
        'import_rolling_mean_24h',
        'load_change_1h', 'load_change_24h', 'import_change_1h'
    ]
    
    for feat in lag_features:
        df_feat[feat] = np.nan
    
    # Create lag features per country (prevents cross-country leakage)
    for country in df_feat['country'].unique():
        mask = df_feat['country'] == country
        
        # Load lags (past values)
        df_feat.loc[mask, 'load_lag_1h'] = df_feat.loc[mask, 'Actual_Load'].shift(1)
        df_feat.loc[mask, 'load_lag_3h'] = df_feat.loc[mask, 'Actual_Load'].shift(3)
        df_feat.loc[mask, 'load_lag_6h'] = df_feat.loc[mask, 'Actual_Load'].shift(6)
        df_feat.loc[mask, 'load_lag_24h'] = df_feat.loc[mask, 'Actual_Load'].shift(24)
        
        # Import lags
        df_feat.loc[mask, 'import_lag_1h'] = df_feat.loc[mask, 'net_imports'].shift(1)
        df_feat.loc[mask, 'import_lag_3h'] = df_feat.loc[mask, 'net_imports'].shift(3)
        df_feat.loc[mask, 'import_lag_24h'] = df_feat.loc[mask, 'net_imports'].shift(24)
        
        # Rolling statistics (moving averages and std dev)
        df_feat.loc[mask, 'load_rolling_mean_6h'] = (
            df_feat.loc[mask, 'Actual_Load'].shift(1).rolling(6, min_periods=1).mean()
        )
        df_feat.loc[mask, 'load_rolling_std_6h'] = (
            df_feat.loc[mask, 'Actual_Load'].shift(1).rolling(6, min_periods=1).std()
        )
        df_feat.loc[mask, 'load_rolling_mean_24h'] = (
            df_feat.loc[mask, 'Actual_Load'].shift(1).rolling(24, min_periods=1).mean()
        )
        df_feat.loc[mask, 'import_rolling_mean_24h'] = (
            df_feat.loc[mask, 'net_imports'].shift(1).rolling(24, min_periods=1).mean()
        )
        
        # Change features (derivatives)
        df_feat.loc[mask, 'load_change_1h'] = df_feat.loc[mask, 'Actual_Load'].diff(1)
        df_feat.loc[mask, 'load_change_24h'] = df_feat.loc[mask, 'Actual_Load'].diff(24)
        df_feat.loc[mask, 'import_change_1h'] = df_feat.loc[mask, 'net_imports'].diff(1)
    
    # Note: Stress lag features will be added after target creation
    print(f"  Created 14 lag features (4 more after targets)")
    
    # Drop first 24 hours per country (insufficient history for lags)
    initial_count = len(df_feat)
    df_feat = df_feat.groupby('country').apply(lambda x: x.iloc[24:]).reset_index(drop=True)
    dropped = initial_count - len(df_feat)
    
    print(f"  Dropped {dropped:,} records (first 24h per country)")
    print(f"  Final: {len(df_feat):,} records with complete lag features")
    
    return df_feat

# ========================================================================
# Apply feature engineering to all datasets
# ========================================================================

print("\n" + "="*70)
print("APPLYING FEATURE ENGINEERING")
print("="*70)

train_engineered = engineer_features(train_df, "Training Set")
val_engineered = engineer_features(val_df, "Validation Set")
test_engineered = engineer_features(test_df, "Test Set")

# ========================================================================
# Verify feature creation
# ========================================================================

print("\n" + "="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)

# Count features by category
base_features = [
    'Actual_Load', 'Forecasted_Load', 'load_rel_error', 'load_difference', 'load_forecast_ratio',
    'net_imports', 'import_magnitude', 'import_dependency_ratio', 'is_importing', 'is_exporting',
    'mean_ssrd', 'mean_wind_speed', 'mean_temperature_c',
    'temp_load_interaction', 'wind_load_interaction', 'solar_load_interaction',
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos',
    'is_weekend', 'is_peak_hour',
    'reserve_margin_ml'
]

lag_features = [
    'load_lag_1h', 'load_lag_3h', 'load_lag_6h', 'load_lag_24h',
    'import_lag_1h', 'import_lag_3h', 'import_lag_24h',
    'load_rolling_mean_6h', 'load_rolling_std_6h', 'load_rolling_mean_24h',
    'import_rolling_mean_24h',
    'load_change_1h', 'load_change_24h', 'import_change_1h'
]

print(f"\nFeature counts:")
print(f"  Base features:  {len(base_features)}")
print(f"  Lag features:   {len(lag_features)} (4 stress lags to be added)")
print(f"  Total so far:   {len(base_features) + len(lag_features)}")

# Check for missing values in engineered features
print(f"\nMissing values in engineered features:")
missing_in_engineered = train_engineered[base_features + lag_features].isnull().sum()
if missing_in_engineered.sum() > 0:
    print("\nFeatures with missing values:")
    for feat in missing_in_engineered[missing_in_engineered > 0].index:
        count = missing_in_engineered[feat]
        pct = (count / len(train_engineered)) * 100
        print(f"  {feat:<30} {count:>7,} ({pct:>5.2f}%)")
    
    print("\nAction: Filling missing values with 0 (safe for lag features)")
    for df in [train_engineered, val_engineered, test_engineered]:
        df[base_features + lag_features] = df[base_features + lag_features].fillna(0)
else:
    print("  No missing values in engineered features")

print("\n" + "="*70)
print("FEATURE ENGINEERING COMPLETE")
print("="*70)
print(f"\nDataset sizes after feature engineering:")
print(f"  Training:   {len(train_engineered):>8,} records")
print(f"  Validation: {len(val_engineered):>8,} records")
print(f"  Test:       {len(test_engineered):>8,} records")
print(f"\nNext step: Add 6-target system to engineered datasets")

In [0]:
# ============================================================================
# Cell 6: Add 6-Target System to Engineered Data + Stress Lag Features
# ============================================================================

print("\n" + "="*70)
print("ADDING 6-TARGET SYSTEM TO ENGINEERED DATASETS")
print("="*70)

def add_6target_system(df, p10_threshold, p90_threshold, p90_magnitude_threshold, dataset_name="Dataset"):
    """
    Add 6-target stress score system to engineered dataset.
    Then create stress-based lag features.
    """
    
    print(f"\n{dataset_name}:")
    print("-" * 70)
    print(f"Initial records: {len(df):,}")
    
    # Create working copy
    df_targets = df.copy()
    
    # Calculate forecast error percentage for targets
    df_targets['forecast_error_pct'] = (
        (df_targets['Actual_Load'] - df_targets['Forecasted_Load']).abs() / 
        df_targets['Forecasted_Load']
    )
    
    # ========================================================================
    # CREATE 6 TARGETS
    # ========================================================================
    
    # T1: Large Forecast Error (>10%) - 25 points
    df_targets['target_T1_large_error'] = (
        df_targets['forecast_error_pct'] > 0.10
    ).astype(int)
    
    # T2: Medium Forecast Error (5-10%) - 10 points
    df_targets['target_T2_medium_error'] = (
        (df_targets['forecast_error_pct'] > 0.05) & 
        (df_targets['forecast_error_pct'] <= 0.10)
    ).astype(int)
    
    # T3: Underestimated Demand (>5% underforecast) - 20 points
    df_targets['target_T3_underestimated'] = (
        ((df_targets['Actual_Load'] - df_targets['Forecasted_Load']) / 
         df_targets['Forecasted_Load'] > 0.05) &
        (df_targets['Forecasted_Load'] < df_targets['Actual_Load'])
    ).astype(int)
    
    # T7: High Exports (<P10) - 10 points
    df_targets['target_T7_high_exports'] = (
        df_targets['net_imports'] < p10_threshold
    ).astype(int)
    
    # T8: High Imports (>P90) - 20 points
    df_targets['target_T8_high_imports'] = (
        df_targets['net_imports'] > p90_threshold
    ).astype(int)
    
    # T9: Extreme Import/Export (|imports| >P90) - 15 points
    df_targets['target_T9_extreme_flow'] = (
        df_targets['net_imports'].abs() > p90_magnitude_threshold
    ).astype(int)
    
    # Calculate stress score
    df_targets['stress_score'] = (
        df_targets['target_T1_large_error'] * 25 +
        df_targets['target_T2_medium_error'] * 10 +
        df_targets['target_T3_underestimated'] * 20 +
        df_targets['target_T7_high_exports'] * 10 +
        df_targets['target_T8_high_imports'] * 20 +
        df_targets['target_T9_extreme_flow'] * 15
    )
    
    # Create blackout risk binary target (threshold = 30)
    df_targets['blackout_risk'] = (df_targets['stress_score'] >= 30).astype(int)
    
    print(f"\n  Target occurrence rates:")
    print(f"    T1 (Large Error):      {df_targets['target_T1_large_error'].mean()*100:5.2f}%")
    print(f"    T2 (Medium Error):     {df_targets['target_T2_medium_error'].mean()*100:5.2f}%")
    print(f"    T3 (Underestimated):   {df_targets['target_T3_underestimated'].mean()*100:5.2f}%")
    print(f"    T7 (High Exports):     {df_targets['target_T7_high_exports'].mean()*100:5.2f}%")
    print(f"    T8 (High Imports):     {df_targets['target_T8_high_imports'].mean()*100:5.2f}%")
    print(f"    T9 (Extreme Flow):     {df_targets['target_T9_extreme_flow'].mean()*100:5.2f}%")
    
    print(f"\n  Stress score statistics:")
    print(f"    Mean:   {df_targets['stress_score'].mean():6.2f}")
    print(f"    Median: {df_targets['stress_score'].median():6.2f}")
    print(f"    Std:    {df_targets['stress_score'].std():6.2f}")
    print(f"    Min:    {df_targets['stress_score'].min():6.0f}")
    print(f"    Max:    {df_targets['stress_score'].max():6.0f}")
    
    print(f"\n  Blackout risk (≥30 points): {df_targets['blackout_risk'].mean()*100:5.2f}%")
    
    # ========================================================================
    # CREATE STRESS LAG FEATURES (4 additional lag features)
    # ========================================================================
    
    print(f"\n  Creating stress lag features...")
    
    # Initialize stress lag columns
    df_targets['stress_lag_1h'] = np.nan
    df_targets['stress_lag_24h'] = np.nan
    df_targets['stress_momentum'] = np.nan
    df_targets['stress_trend_24h'] = np.nan
    
    # Create per country to prevent leakage
    for country in df_targets['country'].unique():
        mask = df_targets['country'] == country
        
        # Stress lags
        df_targets.loc[mask, 'stress_lag_1h'] = df_targets.loc[mask, 'stress_score'].shift(1)
        df_targets.loc[mask, 'stress_lag_24h'] = df_targets.loc[mask, 'stress_score'].shift(24)
        
        # Stress momentum (change from previous hour)
        df_targets.loc[mask, 'stress_momentum'] = df_targets.loc[mask, 'stress_score'].diff(1)
        
        # Stress trend (change from 24h ago)
        df_targets.loc[mask, 'stress_trend_24h'] = df_targets.loc[mask, 'stress_score'].diff(24)
    
    # Drop first row per country (stress lags create NaN in first row)
    initial_count = len(df_targets)
    df_targets = df_targets.groupby('country').apply(lambda x: x.iloc[1:]).reset_index(drop=True)
    dropped = initial_count - len(df_targets)
    
    print(f"  Dropped {dropped:,} records (first row per country for stress lags)")
    print(f"  Final: {len(df_targets):,} records")
    
    # Fill any remaining NaN in stress lag features with 0
    stress_lag_cols = ['stress_lag_1h', 'stress_lag_24h', 'stress_momentum', 'stress_trend_24h']
    df_targets[stress_lag_cols] = df_targets[stress_lag_cols].fillna(0)
    
    return df_targets

# ========================================================================
# Calculate thresholds from TRAINING data
# ========================================================================

print("\nCalculating import/export thresholds from training data...")

p10_imports = train_engineered['net_imports'].quantile(0.10)
p90_imports = train_engineered['net_imports'].quantile(0.90)
p90_magnitude = train_engineered['net_imports'].abs().quantile(0.90)

print(f"  P10 (high exports):      {p10_imports:>10,.2f} MW")
print(f"  P90 (high imports):      {p90_imports:>10,.2f} MW")
print(f"  P90 magnitude (extreme): {p90_magnitude:>10,.2f} MW")

print("\nNote: Using training thresholds for all datasets to prevent data leakage")

# ========================================================================
# Apply 6-target system to all datasets
# ========================================================================

print("\n" + "="*70)
print("ADDING TARGETS TO ALL DATASETS")
print("="*70)

train_final = add_6target_system(
    train_engineered, p10_imports, p90_imports, p90_magnitude, "Training Set"
)

val_final = add_6target_system(
    val_engineered, p10_imports, p90_imports, p90_magnitude, "Validation Set"
)

test_final = add_6target_system(
    test_engineered, p10_imports, p90_imports, p90_magnitude, "Test Set"
)

# ========================================================================
# Final summary
# ========================================================================

print("\n" + "="*70)
print("FINAL FEATURE COUNT")
print("="*70)

# Define complete feature list
FEATURE_COLUMNS = [
    # Core load features (5)
    'Actual_Load', 'Forecasted_Load', 'load_rel_error', 'load_difference', 'load_forecast_ratio',
    
    # Import/export features (5)
    'net_imports', 'import_magnitude', 'import_dependency_ratio', 'is_importing', 'is_exporting',
    
    # Weather features (3)
    'mean_ssrd', 'mean_wind_speed', 'mean_temperature_c',
    
    # Weather interactions (3)
    'temp_load_interaction', 'wind_load_interaction', 'solar_load_interaction',
    
    # Temporal cyclical (6)
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos',
    
    # Temporal binary (2)
    'is_weekend', 'is_peak_hour',
    
    # Reserve margin (1)
    'reserve_margin_ml',
    
    # Lag features - Load (4)
    'load_lag_1h', 'load_lag_3h', 'load_lag_6h', 'load_lag_24h',
    
    # Lag features - Imports (3)
    'import_lag_1h', 'import_lag_3h', 'import_lag_24h',
    
    # Lag features - Stress (2)
    'stress_lag_1h', 'stress_lag_24h',
    
    # Rolling statistics (4)
    'load_rolling_mean_6h', 'load_rolling_std_6h', 'load_rolling_mean_24h', 'import_rolling_mean_24h',
    
    # Change features (5)
    'load_change_1h', 'load_change_24h', 'import_change_1h', 'stress_momentum', 'stress_trend_24h'
]

print(f"\nTotal feature count: {len(FEATURE_COLUMNS)}")
print(f"\nBreakdown:")
print(f"  Core load:          5")
print(f"  Import/export:      5")
print(f"  Weather:            3")
print(f"  Weather interact:   3")
print(f"  Temporal cyclical:  6")
print(f"  Temporal binary:    2")
print(f"  Reserve margin:     1")
print(f"  Lag - Load:         4")
print(f"  Lag - Imports:      3")
print(f"  Lag - Stress:       2")
print(f"  Rolling stats:      4")
print(f"  Change features:    5")
print(f"  " + "-" * 30)
print(f"  TOTAL:             43")

print(f"\nTarget column: stress_score (range: 0-100)")
print(f"Binary target: blackout_risk (threshold: ≥30)")

print(f"\n" + "="*70)
print("FINAL DATASET SIZES")
print("="*70)
print(f"  Training:   {len(train_final):>8,} records")
print(f"  Validation: {len(val_final):>8,} records")
print(f"  Test:       {len(test_final):>8,} records")
print(f"  Total:      {len(train_final) + len(val_final) + len(test_final):>8,} records")

# Verify no missing values in feature columns
print(f"\n" + "="*70)
print("DATA QUALITY CHECK")
print("="*70)

missing_train = train_final[FEATURE_COLUMNS].isnull().sum().sum()
missing_val = val_final[FEATURE_COLUMNS].isnull().sum().sum()
missing_test = test_final[FEATURE_COLUMNS].isnull().sum().sum()

print(f"\nMissing values in feature columns:")
print(f"  Training:   {missing_train:,}")
print(f"  Validation: {missing_val:,}")
print(f"  Test:       {missing_test:,}")

if missing_train + missing_val + missing_test == 0:
    print("\nEXCELLENT: All feature columns are complete!")
else:
    print("\nWARNING: Found missing values - will need to handle before modeling")

print(f"\n" + "="*70)
print("DATA PREPARATION COMPLETE - READY FOR MODELING")
print("="*70)

In [0]:
# ============================================================================
# Cell 7: Data Cleaning and XGBoost Model Training
# ============================================================================

print("\n" + "="*70)
print("PREPARING DATA AND TRAINING MODEL")
print("="*70)

# ========================================================================
# Step 1: Define features and prepare datasets
# ========================================================================

# Define the 43 feature columns
FEATURE_COLUMNS = [
    # Core load features (5)
    'Actual_Load', 'Forecasted_Load', 'load_rel_error', 'load_difference', 'load_forecast_ratio',
    # Import/export features (5)
    'net_imports', 'import_magnitude', 'import_dependency_ratio', 'is_importing', 'is_exporting',
    # Weather features (3)
    'mean_ssrd', 'mean_wind_speed', 'mean_temperature_c',
    # Weather interactions (3)
    'temp_load_interaction', 'wind_load_interaction', 'solar_load_interaction',
    # Temporal cyclical (6)
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos',
    # Temporal binary (2)
    'is_weekend', 'is_peak_hour',
    # Reserve margin (1)
    'reserve_margin_ml',
    # Lag features - Load (4)
    'load_lag_1h', 'load_lag_3h', 'load_lag_6h', 'load_lag_24h',
    # Lag features - Imports (3)
    'import_lag_1h', 'import_lag_3h', 'import_lag_24h',
    # Lag features - Stress (2)
    'stress_lag_1h', 'stress_lag_24h',
    # Rolling statistics (4)
    'load_rolling_mean_6h', 'load_rolling_std_6h', 'load_rolling_mean_24h', 'import_rolling_mean_24h',
    # Change features (5)
    'load_change_1h', 'load_change_24h', 'import_change_1h', 'stress_momentum', 'stress_trend_24h'
]

TARGET_COLUMN = 'stress_score'

print("\nPreparing feature matrices...")
print(f"  Features: {len(FEATURE_COLUMNS)}")
print(f"  Target: {TARGET_COLUMN}")

# Extract features and target
X_train = train_final[FEATURE_COLUMNS].copy()
y_train = train_final[TARGET_COLUMN].copy()

X_val = val_final[FEATURE_COLUMNS].copy()
y_val = val_final[TARGET_COLUMN].copy()

X_test = test_final[FEATURE_COLUMNS].copy()
y_test = test_final[TARGET_COLUMN].copy()

print(f"\nDataset shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")

# ========================================================================
# Step 2: Data cleaning - Handle infinity and extreme values
# ========================================================================

print("\n" + "-"*70)
print("Data Cleaning")
print("-"*70)

def clean_features(X, dataset_name):
    """
    Clean feature data by replacing infinity and extreme values.
    This prevents XGBoost errors during training.
    """
    X_clean = X.copy()
    
    # Replace infinity with NaN
    X_clean = X_clean.replace([np.inf, -np.inf], np.nan)
    
    # Count and fill NaN values with column median
    nan_count = X_clean.isnull().sum().sum()
    if nan_count > 0:
        print(f"  {dataset_name}: Filling {nan_count} NaN/inf values with column medians")
        for col in X_clean.columns:
            if X_clean[col].isnull().any():
                median_val = X_clean[col].median()
                X_clean[col] = X_clean[col].fillna(median_val)
    
    return X_clean

print("\nCleaning datasets...")
X_train_clean = clean_features(X_train, "Training")
X_val_clean = clean_features(X_val, "Validation")
X_test_clean = clean_features(X_test, "Test")

# Verify cleaning
print("\nVerification:")
for name, X in [("Train", X_train_clean), ("Val", X_val_clean), ("Test", X_test_clean)]:
    inf_count = np.isinf(X.values).sum()
    nan_count = X.isnull().sum().sum()
    print(f"  {name}: Inf={inf_count}, NaN={nan_count}")

print("Data cleaning complete")

# ========================================================================
# Step 3: Train XGBoost Regressor
# ========================================================================

print("\n" + "="*70)
print("TRAINING XGBOOST REGRESSOR")
print("="*70)

# Configure model with hyperparameters to prevent overfitting
model = XGBRegressor(
    n_estimators=200,           # Number of boosting rounds
    max_depth=6,                # Maximum tree depth
    learning_rate=0.1,          # Step size shrinkage
    subsample=0.8,              # Row sampling per tree
    colsample_bytree=0.8,       # Column sampling per tree
    min_child_weight=3,         # Minimum sum of weights in child
    random_state=42,            # Reproducibility
    n_jobs=-1,                  # Use all CPU cores
    tree_method='hist',         # Fast histogram algorithm
    objective='reg:squarederror'
)

print("\nModel configuration:")
print(f"  Algorithm:        XGBoost Regressor")
print(f"  n_estimators:     {model.n_estimators}")
print(f"  max_depth:        {model.max_depth}")
print(f"  learning_rate:    {model.learning_rate}")
print(f"  subsample:        {model.subsample}")
print(f"  colsample_bytree: {model.colsample_bytree}")

print("\nTraining model...")

import time
start_time = time.time()

# Train with validation monitoring
model.fit(
    X_train_clean, 
    y_train,
    eval_set=[(X_val_clean, y_val)],
    verbose=False
)

training_time = time.time() - start_time
print(f"Training complete in {training_time:.1f} seconds")

# ========================================================================
# Step 4: Generate predictions
# ========================================================================

print("\n" + "-"*70)
print("Generating Predictions")
print("-"*70)

y_train_pred = model.predict(X_train_clean)
y_val_pred = model.predict(X_val_clean)
y_test_pred = model.predict(X_test_clean)

print("Predictions generated for all datasets")

# ========================================================================
# Step 5: Evaluate model performance
# ========================================================================

print("\n" + "="*70)
print("MODEL PERFORMANCE EVALUATION")
print("="*70)

def evaluate_model(y_true, y_pred, dataset_name):
    """Calculate regression metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    return {
        'dataset': dataset_name,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'y_pred': y_pred
    }

# Evaluate all datasets
train_metrics = evaluate_model(y_train, y_train_pred, "Training")
val_metrics = evaluate_model(y_val, y_val_pred, "Validation")
test_metrics = evaluate_model(y_test, y_test_pred, "Test")

# Display results
print(f"\n{'Dataset':<15} {'MAE':<12} {'RMSE':<12} {'R²':<12}")
print("-" * 52)
for m in [train_metrics, val_metrics, test_metrics]:
    print(f"{m['dataset']:<15} {m['mae']:<12.4f} {m['rmse']:<12.4f} {m['r2']:<12.6f}")

# Analyze generalization
print("\n" + "-"*70)
print("Overfitting Analysis")
print("-"*70)

mae_diff = abs(train_metrics['mae'] - test_metrics['mae'])
r2_diff = abs(train_metrics['r2'] - test_metrics['r2'])

print(f"\nTrain vs Test:")
print(f"  MAE difference:  {mae_diff:.4f} points")
print(f"  R² difference:   {r2_diff:.6f}")

if mae_diff < 1.0 and r2_diff < 0.05:
    print("\nConclusion: Excellent generalization - no overfitting detected")
elif mae_diff < 2.0 and r2_diff < 0.10:
    print("\nConclusion: Good generalization - minimal overfitting")
else:
    print("\nConclusion: Possible overfitting - significant train/test gap")

# ========================================================================
# Summary
# ========================================================================

print("\n" + "="*70)
print("TRAINING SUMMARY")
print("="*70)

print(f"\nModel Details:")
print(f"  Training samples:  {len(X_train_clean):,}")
print(f"  Feature count:     {len(FEATURE_COLUMNS)}")
print(f"  Target:            {TARGET_COLUMN} (0-100 range)")
print(f"  Training time:     {training_time:.1f} seconds")

print(f"\nTest Set Performance:")
print(f"  MAE:  {test_metrics['mae']:.4f} points")
print(f"  RMSE: {test_metrics['rmse']:.4f} points")
print(f"  R²:   {test_metrics['r2']:.6f}")

print("\nNext steps:")
print("  - Feature importance analysis")
print("  - Visualization of predictions")
print("  - Model export for deployment")

In [0]:
# ============================================================================
# Cell 8: Feature Importance Analysis and Visualizations
# ============================================================================

print("\n" + "="*70)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*70)

# ========================================================================
# Extract and analyze feature importance
# ========================================================================

# Get feature importances from the trained model
feature_importance_values = model.feature_importances_

# Create dataframe for analysis
feature_importance_df = pd.DataFrame({
    'feature': FEATURE_COLUMNS,
    'importance': feature_importance_values
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print("-" * 70)
print(f"{'Rank':<6} {'Feature':<35} {'Importance':<12}")
print("-" * 70)

for idx, row in feature_importance_df.head(20).iterrows():
    rank = feature_importance_df.index.get_loc(idx) + 1
    print(f"{rank:<6} {row['feature']:<35} {row['importance']:<12.6f}")

# Categorize features by type
print("\n" + "-"*70)
print("Feature Importance by Category")
print("-"*70)

feature_categories = {
    'Stress Lag': ['stress_lag_1h', 'stress_lag_24h', 'stress_momentum', 'stress_trend_24h'],
    'Load Lag': ['load_lag_1h', 'load_lag_3h', 'load_lag_6h', 'load_lag_24h'],
    'Import Lag': ['import_lag_1h', 'import_lag_3h', 'import_lag_24h'],
    'Core Load': ['Actual_Load', 'Forecasted_Load', 'load_rel_error', 'load_difference', 'load_forecast_ratio'],
    'Import/Export': ['net_imports', 'import_magnitude', 'import_dependency_ratio', 'is_importing', 'is_exporting'],
    'Rolling Stats': ['load_rolling_mean_6h', 'load_rolling_std_6h', 'load_rolling_mean_24h', 'import_rolling_mean_24h'],
    'Temporal': ['hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos', 'is_weekend', 'is_peak_hour'],
    'Weather': ['mean_ssrd', 'mean_wind_speed', 'mean_temperature_c', 'temp_load_interaction', 'wind_load_interaction', 'solar_load_interaction'],
    'Other': ['reserve_margin_ml', 'load_change_1h', 'load_change_24h', 'import_change_1h']
}

category_importance = {}
for category, features in feature_categories.items():
    total_importance = feature_importance_df[feature_importance_df['feature'].isin(features)]['importance'].sum()
    category_importance[category] = total_importance

# Sort and display
category_df = pd.DataFrame(list(category_importance.items()), 
                          columns=['Category', 'Total_Importance']).sort_values('Total_Importance', ascending=False)

print(f"\n{'Category':<20} {'Total Importance':<20} {'Percentage':<15}")
print("-" * 55)
for _, row in category_df.iterrows():
    pct = (row['Total_Importance'] / feature_importance_values.sum()) * 100
    print(f"{row['Category']:<20} {row['Total_Importance']:<20.6f} {pct:<15.2f}%")

# ========================================================================
# Visualizations
# ========================================================================

print("\n" + "="*70)
print("GENERATING VISUALIZATIONS")
print("="*70)

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Top 15 Feature Importances
ax1 = fig.add_subplot(gs[0, :2])
top_15 = feature_importance_df.head(15)
ax1.barh(range(len(top_15)), top_15['importance'], color='steelblue', edgecolor='black')
ax1.set_yticks(range(len(top_15)))
ax1.set_yticklabels(top_15['feature'])
ax1.set_xlabel('Importance Score', fontsize=11)
ax1.set_title('Top 15 Most Important Features', fontsize=13, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(True, alpha=0.3, axis='x')

# 2. Feature Importance by Category
ax2 = fig.add_subplot(gs[0, 2])
ax2.barh(range(len(category_df)), category_df['Total_Importance'], color='coral', edgecolor='black')
ax2.set_yticks(range(len(category_df)))
ax2.set_yticklabels(category_df['Category'])
ax2.set_xlabel('Total Importance', fontsize=11)
ax2.set_title('Importance by Category', fontsize=13, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(True, alpha=0.3, axis='x')

# 3. Actual vs Predicted (Test Set)
ax3 = fig.add_subplot(gs[1, 0])
ax3.scatter(y_test, y_test_pred, alpha=0.3, s=1, color='steelblue')
ax3.plot([0, 80], [0, 80], 'r--', linewidth=2, label='Perfect Prediction')
ax3.set_xlabel('Actual Stress Score', fontsize=11)
ax3.set_ylabel('Predicted Stress Score', fontsize=11)
ax3.set_title(f'Actual vs Predicted - Test Set\n(R²={test_metrics["r2"]:.6f})', 
              fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Residuals Distribution
ax4 = fig.add_subplot(gs[1, 1])
residuals_test = y_test - y_test_pred
ax4.hist(residuals_test, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax4.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
ax4.set_xlabel('Residual (Actual - Predicted)', fontsize=11)
ax4.set_ylabel('Frequency', fontsize=11)
ax4.set_title(f'Residuals Distribution - Test Set\nMean={residuals_test.mean():.4f}, Std={residuals_test.std():.4f}', 
              fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Residuals vs Predicted
ax5 = fig.add_subplot(gs[1, 2])
ax5.scatter(y_test_pred, residuals_test, alpha=0.3, s=1, color='steelblue')
ax5.axhline(0, color='red', linestyle='--', linewidth=2)
ax5.set_xlabel('Predicted Stress Score', fontsize=11)
ax5.set_ylabel('Residual', fontsize=11)
ax5.set_title('Residual Plot - Test Set', fontsize=12, fontweight='bold')
ax5.grid(True, alpha=0.3)

# 6. Prediction Distribution Comparison
ax6 = fig.add_subplot(gs[2, 0])
ax6.hist(y_test, bins=30, alpha=0.6, label='Actual', color='blue', edgecolor='black')
ax6.hist(y_test_pred, bins=30, alpha=0.6, label='Predicted', color='red', edgecolor='black')
ax6.axvline(30, color='orange', linestyle='--', linewidth=2, label='Blackout Threshold')
ax6.set_xlabel('Stress Score', fontsize=11)
ax6.set_ylabel('Frequency', fontsize=11)
ax6.set_title('Distribution: Actual vs Predicted', fontsize=12, fontweight='bold')
ax6.legend()
ax6.grid(True, alpha=0.3)

# 7. Performance Across Datasets
ax7 = fig.add_subplot(gs[2, 1])
datasets = ['Train', 'Val', 'Test']
mae_values = [train_metrics['mae'], val_metrics['mae'], test_metrics['mae']]
rmse_values = [train_metrics['rmse'], val_metrics['rmse'], test_metrics['rmse']]

x = np.arange(len(datasets))
width = 0.35

ax7.bar(x - width/2, mae_values, width, label='MAE', color='steelblue', edgecolor='black')
ax7.bar(x + width/2, rmse_values, width, label='RMSE', color='coral', edgecolor='black')
ax7.set_ylabel('Error (points)', fontsize=11)
ax7.set_title('Performance Across Datasets', fontsize=12, fontweight='bold')
ax7.set_xticks(x)
ax7.set_xticklabels(datasets)
ax7.legend()
ax7.grid(True, alpha=0.3, axis='y')

# 8. R² Comparison
ax8 = fig.add_subplot(gs[2, 2])
r2_values = [train_metrics['r2'], val_metrics['r2'], test_metrics['r2']]
bars = ax8.bar(datasets, r2_values, color='green', edgecolor='black', alpha=0.7)
ax8.set_ylabel('R² Score', fontsize=11)
ax8.set_title('R² Score Comparison', fontsize=12, fontweight='bold')
ax8.set_ylim([0.999, 1.0])
ax8.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, r2_values):
    height = bar.get_height()
    ax8.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.6f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('European Power Grid Stress Prediction - Model Analysis', 
             fontsize=16, fontweight='bold', y=0.995)

plt.show()

print("\nVisualizations complete")

# ========================================================================
# Key Insights
# ========================================================================

print("\n" + "="*70)
print("KEY INSIGHTS")
print("="*70)

print("\n1. Model Performance:")
print(f"   - Achieved R² of {test_metrics['r2']:.6f} on test set")
print(f"   - Mean prediction error: {test_metrics['mae']:.4f} points")
print(f"   - Excellent generalization (no overfitting)")

print("\n2. Most Important Features:")
top_5 = feature_importance_df.head(5)
for idx, row in top_5.iterrows():
    rank = feature_importance_df.index.get_loc(idx) + 1
    print(f"   {rank}. {row['feature']} ({row['importance']:.6f})")

print("\n3. Feature Categories:")
print(f"   - Most important: {category_df.iloc[0]['Category']} ({category_df.iloc[0]['Total_Importance']:.4f})")
print(f"   - Second: {category_df.iloc[1]['Category']} ({category_df.iloc[1]['Total_Importance']:.4f})")

print("\n4. Prediction Quality:")
print(f"   - Residuals centered at {residuals_test.mean():.4f} (near zero)")
print(f"   - Residual std: {residuals_test.std():.4f} points")
print(f"   - Model captures both low and high stress events accurately")

print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)

In [0]:
"""
==============================================================================
CELL 9: SAVE MODEL AND ARTIFACTS FOR STREAMLIT
==============================================================================
Save the trained model and all necessary files for deployment
==============================================================================
"""

import pickle
import json
import os

print("="*80)
print("SAVING MODEL FOR STREAMLIT DEPLOYMENT")
print("="*80)

# ============================================================
# 9.1: CREATE SAVE DIRECTORY
# ============================================================
print("\n[9.1 CREATING SAVE DIRECTORY]")
print("-"*80)

save_dir = '/tmp/grid_stress_final_model'
os.makedirs(save_dir, exist_ok=True)
print(f"Directory created: {save_dir}")

# ============================================================
# 9.2: SAVE THE TRAINED MODEL
# ============================================================
print("\n[9.2 SAVING TRAINED MODEL]")
print("-"*80)

# The model should be named something like xgb_regressor or xgb_regressor_6target
# Let's check what model variables exist
model_names = ['xgb_regressor_6target', 'xgb_regressor', 'model', 'final_model']
model_to_save = None

for name in model_names:
    try:
        model_to_save = eval(name)
        print(f"Found model: {name}")
        break
    except NameError:
        continue

if model_to_save is not None:
    model_path = f'{save_dir}/grid_stress_model.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump(model_to_save, f)
    
    file_size = os.path.getsize(model_path) / (1024*1024)
    print(f"✓ Model saved: {model_path}")
    print(f"  Size: {file_size:.2f} MB")
else:
    print("✗ Model not found in memory")
    print("Available variables:", [v for v in dir() if 'model' in v.lower()])

# ============================================================
# 9.3: SAVE FEATURE CONFIGURATION
# ============================================================
print("\n[9.3 SAVING FEATURE CONFIGURATION]")
print("-"*80)

# Check if available_features exists
try:
    feature_list = available_features
    print(f"Found {len(feature_list)} features")
except NameError:
    # If not, try to get from X_train columns
    try:
        feature_list = X_train.columns.tolist()
        print(f"Retrieved {len(feature_list)} features from X_train")
    except:
        print("✗ Could not find feature list")
        feature_list = []

if feature_list:
    feature_config = {
        'feature_names': feature_list,
        'n_features': len(feature_list),
        'model_type': 'XGBoost Regressor',
        'target': 'stress_score (0-100)',
        'blackout_threshold': 30,
        'training_date': '2025-11-25'
    }
    
    config_path = f'{save_dir}/feature_config.json'
    with open(config_path, 'w') as f:
        json.dump(feature_config, f, indent=2)
    print(f"✓ Feature config saved: {config_path}")

# ============================================================
# 9.4: SAVE TARGET DEFINITIONS
# ============================================================
print("\n[9.4 SAVING TARGET DEFINITIONS]")
print("-"*80)

target_definitions = {
    'targets': [
        {'id': 'T1', 'name': 'Large Forecast Error', 'weight': 25, 'threshold': '>10%'},
        {'id': 'T2', 'name': 'Medium Forecast Error', 'weight': 10, 'threshold': '5-10%'},
        {'id': 'T3', 'name': 'Underestimated Demand', 'weight': 20, 'threshold': '>5%'},
        {'id': 'T7', 'name': 'High Exports', 'weight': 10, 'threshold': '<P10'},
        {'id': 'T8', 'name': 'High Imports', 'weight': 20, 'threshold': '>P90'},
        {'id': 'T9', 'name': 'Extreme Import/Export', 'weight': 15, 'threshold': '>P90 abs'}
    ],
    'max_score': 100,
    'blackout_threshold': 30,
    'categories': {
        'NORMAL': '0-29',
        'WARNING': '30-59',
        'CRITICAL': '60-100'
    }
}

targets_path = f'{save_dir}/target_definitions.json'
with open(targets_path, 'w') as f:
    json.dump(target_definitions, f, indent=2)
print(f"✓ Target definitions saved: {targets_path}")

# ============================================================
# 9.5: SAVE PERFORMANCE METRICS
# ============================================================
print("\n[9.5 SAVING PERFORMANCE METRICS]")
print("-"*80)

performance = {
    'test_set': {
        'r2_score': 0.999878,
        'mae': 0.0247,
        'rmse': 0.1670
    },
    'training_info': {
        'training_samples': 385973,
        'validation_samples': 111142,
        'test_samples': 53047,
        'features_used': len(feature_list) if feature_list else 43
    }
}

perf_path = f'{save_dir}/model_performance.json'
with open(perf_path, 'w') as f:
    json.dump(performance, f, indent=2)
print(f"✓ Performance metrics saved: {perf_path}")

# ============================================================
# 9.6: PRINT JSON FILES FOR COPY-PASTE
# ============================================================
print("\n[9.6 FILE CONTENTS FOR STREAMLIT]")
print("="*80)

print("\n--- target_definitions.json ---")
print(json.dumps(target_definitions, indent=2))

print("\n--- model_performance.json ---")
print(json.dumps(performance, indent=2))

# ============================================================
# 9.7: SUMMARY
# ============================================================
print("\n[9.7 FILES SAVED]")
print("="*80)

print(f"\nAll files saved to: {save_dir}")
print("\nFiles created:")
print("  1. grid_stress_model.pkl")
print("  2. feature_config.json")
print("  3. target_definitions.json")
print("  4. model_performance.json")

print("\n" + "="*80)
print("MODEL SAVED - READY FOR STREAMLIT!")
print("="*80)

print("""
NEXT STEP: Start NEW conversation for Streamlit with this info:

"I have a trained European power grid stress predictor:
- Model: XGBoost Regressor (R² = 0.9999, MAE = 0.02)
- Input: 43 features (load, imports, weather, lag features)
- Output: Stress score 0-100
- 6-target system: T1-T9 with weights 25/20/15/10

Need Streamlit dashboard with interactive sliders.
Help me build it!"
""")

In [0]:
"""
==============================================================================
CELL 10: SAVE MODEL FILES TO WORKSPACE (VISIBLE IN FILE BROWSER)
==============================================================================
Save model and configs to your workspace directory for easy access
==============================================================================
"""

import pickle
import json
import os

print("="*80)
print("SAVING MODEL TO WORKSPACE DIRECTORY")
print("="*80)

# ============================================================
# 10.1: DETERMINE CURRENT NOTEBOOK DIRECTORY
# ============================================================
print("\n[10.1 FINDING NOTEBOOK DIRECTORY]")
print("-"*80)

# Get current notebook path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    notebook_dir = '/Workspace' + '/'.join(notebook_path.split('/')[:-1])
    print(f"Notebook directory: {notebook_dir}")
except:
    # Fallback to manual path based on your screenshot
    notebook_dir = '/Workspace/Repos/peterbranch/energy-grid-load-processing/energy_grid_load_processing'
    print(f"Using manual path: {notebook_dir}")

# Create model_files subdirectory
save_dir = f'{notebook_dir}/model_files'
os.makedirs(save_dir, exist_ok=True)
print(f"✓ Save directory created: {save_dir}")

# ============================================================
# 10.2: SAVE MODEL
# ============================================================
print("\n[10.2 SAVING MODEL]")
print("-"*80)

model_path = f'{save_dir}/grid_stress_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

file_size = os.path.getsize(model_path) / (1024*1024)
print(f"✓ Model saved: grid_stress_model.pkl")
print(f"  Size: {file_size:.2f} MB")
print(f"  Location: {model_path}")

# ============================================================
# 10.3: SAVE FEATURE NAMES
# ============================================================
print("\n[10.3 SAVING FEATURE CONFIGURATION]")
print("-"*80)

feature_names = X_train.columns.tolist()

feature_config = {
    'feature_names': feature_names,
    'n_features': len(feature_names),
    'model_type': 'XGBoost Regressor',
    'target': 'stress_score (0-100)',
    'blackout_threshold': 30
}

config_path = f'{save_dir}/feature_config.json'
with open(config_path, 'w') as f:
    json.dump(feature_config, f, indent=2)
print(f"✓ Feature config saved: feature_config.json")

# ============================================================
# 10.4: SAVE TARGET DEFINITIONS
# ============================================================
print("\n[10.4 SAVING TARGET DEFINITIONS]")
print("-"*80)

target_definitions = {
    'targets': [
        {'id': 'T1', 'name': 'Large Forecast Error', 'weight': 25, 'threshold': '>10%'},
        {'id': 'T2', 'name': 'Medium Forecast Error', 'weight': 10, 'threshold': '5-10%'},
        {'id': 'T3', 'name': 'Underestimated Demand', 'weight': 20, 'threshold': '>5%'},
        {'id': 'T7', 'name': 'High Exports', 'weight': 10, 'threshold': '<P10'},
        {'id': 'T8', 'name': 'High Imports', 'weight': 20, 'threshold': '>P90'},
        {'id': 'T9', 'name': 'Extreme Import/Export', 'weight': 15, 'threshold': '>P90 abs'}
    ],
    'max_score': 100,
    'blackout_threshold': 30,
    'categories': {
        'NORMAL': '0-29',
        'WARNING': '30-59',
        'CRITICAL': '60-100'
    }
}

targets_path = f'{save_dir}/target_definitions.json'
with open(targets_path, 'w') as f:
    json.dump(target_definitions, f, indent=2)
print(f"✓ Target definitions saved: target_definitions.json")

# ============================================================
# 10.5: SAVE PERFORMANCE METRICS
# ============================================================
print("\n[10.5 SAVING PERFORMANCE METRICS]")
print("-"*80)

performance = {
    'test_set': {
        'r2_score': 0.999878,
        'mae': 0.0247,
        'rmse': 0.1670
    },
    'training_info': {
        'training_samples': 385973,
        'validation_samples': 111142,
        'test_samples': 53047,
        'features_used': 43
    }
}

perf_path = f'{save_dir}/model_performance.json'
with open(perf_path, 'w') as f:
    json.dump(performance, f, indent=2)
print(f"✓ Performance metrics saved: model_performance.json")

# ============================================================
# 10.6: LIST ALL SAVED FILES
# ============================================================
print("\n[10.6 FILES SAVED]")
print("="*80)

print(f"\n✓ All files saved to: {save_dir}")
print("\nFiles created (visible in file browser):")
print("  1. grid_stress_model.pkl        (0.84 MB)")
print("  2. feature_config.json")
print("  3. target_definitions.json")
print("  4. model_performance.json")

print("\nTO VIEW FILES:")
print(f"1. In left sidebar, navigate to: {save_dir}")
print("2. You'll see a 'model_files' folder")
print("3. Right-click any file → Download")

print("\n" + "="*80)
print("FILES SAVED TO WORKSPACE - VISIBLE IN FILE BROWSER!")
print("="*80)

# Verify files exist
print("\nVerifying files...")
for filename in ['grid_stress_model.pkl', 'feature_config.json', 'target_definitions.json', 'model_performance.json']:
    filepath = f'{save_dir}/{filename}'
    if os.path.exists(filepath):
        print(f"  ✓ {filename}")
    else:
        print(f"  ✗ {filename} - NOT FOUND")

print("\n" + "="*80)
print("READY FOR STREAMLIT!")
print("="*80)